In [18]:
from unfolding_helper import *
from shapely.geometry import *

def get_unfolding(mesh, cutted_faces):
    tree = node(cutted_faces[0])
    tree.add_child(cutted_faces[1])
    return tree.unfold(mesh)

def starting_point_3d_2_2d(mesh, face_id, starting_point_3d):
    map_to_2d = lambda points: mesh.get_2d_projection(mesh.face_handle(face_id)).dot(points)
    return map_to_2d(starting_point_3d)

# Simple Vector Class
class Vec:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def len(self):
        return sqrt(self.x**2 + self.y**2 + self.z**2)

    def normalize(self):
        l = self.len()
        return Vec(self.x/l, self.y/l, self.z/l)

    def __add__(self, v):
        return Vec(self.x + v.x, self.y + v.y, self.z + v.z)

    def __mul__(self, s):
        return Vec(self.x * s, self.y * s, self.z * s)

    def __sub__(self, v):
        return Vec(self.x - v.x, self.y - v.y, self.z - v.z)

In [19]:
mesh = Mesh('cube.obj')
    
#list of faces on the shortest path
cut_faces = [5, 2]
    
end_point_id = 2
starting_point_3d = (.5, .5, .1)
    
#unfold the faces from cutted_faces as shown in the previous lectures
unfold = get_unfolding(mesh, cut_faces)
    
#list of edge_ids that possibly are being cut by the shortest path
cut_edges = get_cutted_edges(mesh, cut_faces)
print('Cutted Edges:', cut_edges)
    
#create a Hashmap that maps vertex_id to 2d coordinates
points_2d = get_2d_points(mesh, unfold, cut_faces)
    
#now we can get the 2d projection of the end point
end_point_2d = points_2d[end_point_id]
    
#project starting point from 3d to 2d space using the first Matrix Multiplication in the unfold function
starting_point_2d = starting_point_3d_2_2d(mesh, cut_faces[0], starting_point_3d)
    
print('Shortest path in 2d will start at', starting_point_2d, 'and end at', end_point_2d)

Cutted Edges: [1, 6, 7, 8, 5, 11, 9]
Shortest path in 2d will start at [0.5 0.5] and end at [0. 2.]


In [20]:
#List of edges represented as vertex_ids
#Example: [(0, 1), (1, 2), (2, 3), (3, 0), (0, 4), (4, 5), (5, 1), (5, 6), (6, 2), (6, 7), (7, 3), (7, 4)]
#now we can get the vertecies of an edge by edges_as_vertex_ids[edge_id]
edge_as_vertex_ids = [tuple(e) for e in mesh.edge_vertex_indices()]

In [21]:
#list to store generated lines
lines = []

#for each edge_id we get the right vertex_ids 
for e in cut_edges:
    vertex_ids = edge_as_vertex_ids[e]
    
    #Create a LineString out of 2d coordinates of these verticies
    nl = LineString((points_2d[vertex_ids[0]], points_2d[vertex_ids[1]]))
    
    #Append this line to lines
    lines.append(nl)

# Create a LineString out of the 2d starting and the 2d end point
main_line = LineString((starting_point_2d, end_point_2d))

In [22]:


class Insertion_Vertex:
    def __init__(self, edge_id, dist):
        self.edge_id = edge_id
        self.distance = dist
    
    def calc_pos(self, edge):
        #expecting the edge to already be in Vector form
        return (edge[1] - edge[0]).normalize() * (self.distance) + edge[0]


In [23]:
new_vertices = []
for i, l in enumerate(lines):   
    #intersect mainline with current line l
    p = main_line.intersection(l)  
    
    #if there is an intersection get the coordinates of p and store them
    if p: 
        #turn p into a vector
        p = tuple(p.coords)[0]
        p = Vec(p[0], p[1], 0)
        
        #turn the first point in the line into a vector
        A = tuple(l.coords)[0]
        A = Vec(A[0], A[1], 0)
        
        #calculate the distance betwenn p and A
        dist = (p - A).len()
        
        #turn p into an Insertion_Vertex
        p = Insertion_Vertex(i, dist)
        new_vertices.append(p)

In [24]:
#List of 3d Coordinates for every vertex in the mesh
# use "points_3d[vertex_id]" to get the coordinates of a certain point
points_3d = [tuple(p) for p in mesh.points()]

#list to store the 3d Vertices
new_vertices = []
for v in new_vertices:
    #get the edge as tuple of vertex_ids 
    edge = edge_as_vertex_ids[v.edge_id]
    
    #turn the edge to tuple of 3d coordinates represented as a vector
    edge = [Vec(*points_3d[id]) for id in edge]
    
    # Calculate the 3d position of the new Vertex and append it to new_vertices_3d
    v = v.calc_pos(edge)
    new_vertices.append(v)

#print all new vertices
for p in new_vertices:
    print(p.x, p.y, p.z)

In [25]:
# This is how you would define a mesh in openmesh:
#from openmesh import *
#mesh = PolyMesh()
 
#A = (1, 0, 0)
#B = (1, 1, 0)
#C = (0, 1, 0)
#D = (0, 0, 0)
 
#top point
#E = (1, 0, 1)
#F = (1, 1, 1)
#G = (0, 1, 1)
#H = (0, 0, 1)

#A = mesh.add_vertex(A)
#B = mesh.add_vertex(B)
#C = mesh.add_vertex(C)
#D = mesh.add_vertex(D)
#E = mesh.add_vertex(E)
#F = mesh.add_vertex(F)
#G = mesh.add_vertex(G)
#H = mesh.add_vertex(H)

#Order in which vertices are inserted is imported --> clockwise
#bottom = mesh.add_face(A, B, C, D)
#front = mesh.add_face(A, E, F, B)
#right = mesh.add_face(B, F, G, C)
#back = mesh.add_face(D, C, G, H)
#left = mesh.add_face(A, D, H, E)
#top = mesh.add_face(E, H, G, F)
  
#write_mesh('test.obj', mesh)